In [1]:
import pandas as pd
from main import Platform, Coin, Transaction

import defi_tools as dft

portfolio = pd.read_csv('portfolio.csv')
portfolio.fillna(0, inplace=True)
portfolio.drop('Unnamed: 0', axis=1, inplace=True)
portfolio.rename(columns={'Unnamed: 1':'id'}, inplace=True)
portfolio.rename(columns={'Coinbase Pro':'Coinbase_Pro'}, inplace=True)
portfolio.set_index('id', inplace=True)


In [2]:
platforms = {k:'' for k in portfolio.columns}
ids = portfolio.index.tolist()
prices = {i:dft.geckoPrice(i, 'usd')[i]['usd'] for i in ids} 
tickers = {i:dft.geckoGetSymbol(i) for i in ids}

In [3]:
for p in platforms.keys():
    platforms[p] = Platform(p)
    for i in ids:
        coin = Coin(coin=i, ticker=tickers[i], price=float(prices[i]), holdings=portfolio.loc[i][p])
        platforms[p].add(coin)

In [4]:
platforms['Coinbase_Pro'].tokens(token='ethereum')

0.35

In [5]:
platforms['Coinbase_Pro'].show()

Total Value: 13513.60611138


ValueError: could not convert string to float: 'ETH'

In [ ]:
cb_pro = platforms['Coinbase_Pro']
cb = platforms['Coinbase']
Transaction.transfer(origin=cb_pro, destination=cb, token='ethereum', amount=0.01, fee=0.0001)
platforms['Coinbase_Pro'].to_df()

Transferred 0.01 ETH from Coinbase_Pro to Coinbase


,Coin,Holdings
0,ETH,3.399000e-01
1,USDC,1.000000e+00
2,BTC,7.000000e-02
3,ADA,2.755000e+03
4,LINK,9.998000e+01
5,DOT,5.200000e+01
6,UNI,8.000000e+01
7,LRC,3.370000e+02
8,CRO,1.030000e+00
9,USDT,6.270400e-02


In [ ]:
platforms['Coinbase'].to_df()

,Coin,Holdings
0,ETH,0.010000
1,USDC,0.003664
2,BTC,0.001958
3,DOGE,265.480000
4,XLM,70.190000
5,BAT,10.380000
6,COMP,0.030063


In [ ]:
cb_pro.add(Coin(coin='USD', ticker='USD', price=1, holdings=2000))

In [ ]:
Transaction.transact(cb_pro, 'terra-luna', amount=1000, proceeds=1000, type='buy')

Bought 1000 LUNA.


In [ ]:
cb_pro.to_df()

,Coin,Holdings
0,ETH,3.399000e-01
1,USDC,1.000000e+00
2,BTC,7.000000e-02
3,ADA,2.755000e+03
4,LINK,9.998000e+01
5,DOT,5.200000e+01
6,UNI,8.000000e+01
7,LRC,3.370000e+02
8,CRO,1.030000e+00
9,LUNA,1.000000e+03


In [ ]:
Transaction.transact(cb_pro, 'terra-luna', amount=1000, proceeds=1000, type='sell')

Sold 1000 LUNA.


In [ ]:
cb_pro.to_df()

,Coin,Holdings
0,ETH,3.399000e-01
1,USDC,1.000000e+00
2,BTC,7.000000e-02
3,ADA,2.755000e+03
4,LINK,9.998000e+01
5,DOT,5.200000e+01
6,UNI,8.000000e+01
7,LRC,3.370000e+02
8,CRO,1.030000e+00
9,USDT,6.270400e-02


In [ ]:
all = False
token = 'ETH'
df = pd.read_csv('transactions.csv')
if all:
    token = df['tokens'].unique()
    t = 'All'
t = token
avgB = df[(df.token.contains(token))&(df.type == 'buy')]['value'].mean() 
avgS = df[(df.token.contains(token))&(df.type == 'sell')]['value'].mean()
profit = avgS - avgB 
print(f'{t}| average buy: {avgB}, average sell: {avgS}, profit: {profit} ({profit/avgB}%)' )


FileNotFoundError: [Errno 2] No such file or directory: 'transactions.csv'